用单层神经网络实现手写数字识别，输入是[55000,784]输出是[55000,10]
网络结构分为2层，输入层和输出层，无隐层

1，数据准备

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.test.images.shape,mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


2,准备好placeholder

In [4]:
batch_size = 256
x = tf.placeholder(tf.float32,[batch_size,784],name="x")
y_ = tf.placeholder(tf.float32,[batch_size,10],name="y")

3,初始化参数/权重

In [5]:
w = tf.Variable(tf.zeros([784,10]),name="w")
b = tf.Variable(tf.zeros([10]),name="b")

4,拿到每个类别的score

In [6]:
logits = tf.matmul(x,w) + b

5,计算多分类的softmax的loss function

In [8]:
#交叉熵损失
entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y_,name="loss")
#求平均值
loss = tf.reduce_mean(entropy)

6,准备好optimizer

In [9]:
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
7,在session里执行graph里定义的运算

In [14]:
n_epoch = 30
init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs/logistic_rec",sess.graph)
    sess.run(init)
    n_batch = int(mnist.train.num_examples/batch_size)
    for i in range(n_epoch):
        total_loss = 0
        for _ in range(n_batch):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={x:x_batch,y_:y_batch})
            total_loss += loss_batch
        print("epoch {0}:{1}".format(i,total_loss))
    print("optimizer finished")
    
    #测试模型
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(y_,1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
    
    n_batch = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batch):
        x_batch,y_batch = mnist.train.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy],feed_dict={x:x_batch,y_:y_batch})
        total_correct_preds += accuracy_batch[0]
    print("accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    
    writer.close()
        
    

epoch 0:84.77315130829811
epoch 1:62.57736298441887
epoch 2:60.10866838693619
epoch 3:57.68238312005997
epoch 4:56.61776652932167
epoch 5:57.062183275818825
epoch 6:55.28260776400566
epoch 7:54.37119561433792
epoch 8:55.99107585847378
epoch 9:55.31081907451153
epoch 10:52.587845996022224
epoch 11:54.12072789669037
epoch 12:53.79538296163082
epoch 13:54.49588494002819
epoch 14:53.36794748157263
epoch 15:53.194341257214546
epoch 16:52.82488125562668
epoch 17:52.953132182359695
epoch 18:52.81838233023882
epoch 19:52.01245100796223
epoch 20:52.00089593231678
epoch 21:53.04213845729828
epoch 22:51.77782931923866
epoch 23:53.33184579014778
epoch 24:51.30111961811781
epoch 25:51.6232640221715
epoch 26:51.49302378296852
epoch 27:52.50800713151693
epoch 28:52.676026947796345
epoch 29:50.445825740695
optimizer finished
accuracy 0.9334
